# General Statistics

In [4]:
import pandas as pd
import os
from helpers import *
from sklearn.impute import SimpleImputer
from dataset_path import output_path,mimic_iv_path
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from tabulate import tabulate
path = output_path
df_train = pd.read_csv(os.path.join(path, 'train_new.csv'))
df_test = pd.read_csv(os.path.join(path, 'test_new.csv'))


In [19]:
# Set the display option to show all rows
pd.set_option('display.max_columns', None)

In [5]:
# Assuming df_train and df_test have the same columns
df_master = pd.concat([df_train, df_test], ignore_index=True)

In [25]:
# Count the number of people with disposition = HOME
home_disposition_count = (df_master['disposition'] == 'HOME').sum()

# Print the result
print("Number of people with disposition = HOME:", home_disposition_count)


Number of people with disposition = HOME: 240323


In [27]:
df_master['disposition'].unique()

array(['HOME', 'ADMITTED', 'TRANSFER', 'ELOPED',
       'LEFT WITHOUT BEING SEEN', 'LEFT AGAINST MEDICAL ADVICE', 'OTHER',
       'EXPIRED'], dtype=object)

In [21]:
df_master

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,triage_pain,triage_acuity,chiefcomplaint,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness,cci_MI,cci_CHF,cci_PVD,cci_Stroke,cci_Dementia,cci_Pulmonary,cci_Rheumatic,cci_PUD,cci_Liver1,cci_DM1,cci_DM2,cci_Paralysis,cci_Renal,cci_Cancer1,cci_Liver2,cci_Cancer2,cci_HIV,eci_CHF,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_PVD,eci_HTN1,eci_HTN2,eci_Paralysis,eci_NeuroOther,eci_Pulmonary,eci_DM1,eci_DM2,eci_Hypothyroid,eci_Renal,eci_Liver,eci_PUD,eci_HIV,eci_Lymphoma,eci_Tumor2,eci_Tumor1,eci_Rheumatic,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon,time_to_microbio_event_hrs,microbiology_event,time_to_med_event_hr,med_event,triage_temperature_knn,triage_heartrate_knn,triage_resprate_knn,triage_o2sat_knn,triage_sbp_knn,triage_dbp_knn,triage_pain_knn,ed_temperature_last_knn,ed_heartrate_last_knn,ed_resprate_last_knn,ed_o2sat_last_knn,ed_sbp_last_knn,ed_dbp_last_knn,ed_pain_last_knn,triage_temperature_median,triage_heartrate_median,triage_resprate_median,triage_o2sat_median,triage_sbp_median,triage_dbp_median,triage_pain_median,ed_temperature_last_median,ed_heartrate_last_median,ed_resprate_last_median,ed_o2sat_last_median,ed_sbp_last_median,ed_dbp_last_median,ed_pain_last_median,triage_MAP,score_CCI,score_CART,score_REMS,score_NEWS,score_NEWS2,score_MEWS
0,183244,14328106,NaN,32336955,2159-11-08 15:13:00,2159-11-09 01:47:00,F,WHITE,WALK IN,HOME,75,2159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159,75,False,0 days 10:34:00,NaN,NaN,False,False,False,0,0,0,NaN,NaN,False,0,0,0,0,0,0,10.57,NaN,NaN,38.055556,65.0,18.0,100.0,135.0,59.0,0.0,2.0,"Dizziness, Unsteady gait",False,False,False,False,False,False,False,False,False,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.444444,53.0,16.0,98.0,130.0,68.0,0.0,5.0,9.0,NaN,0.0,NaN,1.0,38.055556,65.0,18.0,100.0,135.0,59.0,0.0,36.444444,53.0,16.0,98.0,130.0,68.0,0.0,38.055556,65.0,18.0,100.0,135.0,59.0,0.0,36.444444,53.0,16.0,98.0,130.0,68.0,0.0,84.333333,3,9,8,1,1,1
1,376935,18860711,NaN,35614382,2159-10-24 06:19:00,2159-10-24 14:31:00,F,WHITE,WALK IN,HOME,28,2159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159,28,False,0 days 08:12:00,NaN,NaN,False,False,False,1,1,1,NaN,NaN,False,0,0,0,0,0,0,8.20,NaN,NaN,36.500000,89.0,18.0,97.0,103.0,46.0,10.0,3.0,"Abd pain, n/v/d",False,True,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.722222,83.0,16.0,99.0,112.0,51.0,4.0,1.0,5.0,NaN,0.0,NaN,1.0,36.500000,89.0,18.0,97.0,103.0,46.0,10.0,36.722222,83.0,16.0,99.0,112.0,51.0,4.0,36.500000,89.0,18.0,97.0,103.0,46.0,10.0,36.722222,83.0,16.0,99.0,112.0,51.0,4.0,65.000000,0,4,2,1,1,1
2,291579,16860825,NaN,37485359,2203-01-07 17:45:00,2203-01-08 07:00:00,F,BLACK/AFRICAN AMERICAN,AMBULANCE,HOME,52,2194,NaN,NaN,NaN,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,NaN,2203,61,False,0 days 13:15:00,NaN,NaN,False,False,False,0,0,5,2203-01-12 22:26:00,4 days 15:26:00,False,0,0,2,0,0,0,13.25,NaN,4.64,36.888889

In [30]:
# Rename the 'microbiology_event' column to 'microbio_event'
#df_master.rename(columns={'microbiology_event': 'microbio_event'}, inplace=True)


## Overall

In [81]:
continuous_vars = ["age", "triage_temperature_knn", "triage_heartrate_knn", "triage_resprate_knn",
                    "triage_o2sat_knn", "triage_sbp_knn", "triage_dbp_knn", "triage_pain_knn",
                    "ed_temperature_last_knn", "ed_heartrate_last_knn", "ed_resprate_last_knn",
                    "ed_o2sat_last_knn", "ed_sbp_last_knn", "ed_dbp_last_knn", "ed_los_hours",
                       "n_ed_30d", "n_ed_90d", "n_ed_365d", "n_hosp_30d", "n_hosp_90d",
                    "n_hosp_365d", "n_icu_30d", "n_icu_90d", "n_icu_365d", "triage_acuity",]

# Create a table to store the results
table = []

# Calculate mean and standard deviation for each continuous variable
for variable in continuous_vars:
    # Calculate overall mean and standard deviation
    overall_mean = df_master[variable].mean()
    overall_std = df_master[variable].std()

    # Append results to the table
    table.append([variable, "Overall", f"{overall_mean:.2f}", f"{overall_std:.2f}"])

# Display the table
headers = ["Variable", "Outcome", "Mean", "Standard Deviation"]
print(tabulate(table, headers=headers, tablefmt="pretty"))

+-------------------------+---------+--------+--------------------+
|        Variable         | Outcome |  Mean  | Standard Deviation |
+-------------------------+---------+--------+--------------------+
|           age           | Overall | 52.83  |       20.61        |
| triage_temperature_knn  | Overall | 36.71  |        0.54        |
|  triage_heartrate_knn   | Overall | 85.02  |       17.50        |
|   triage_resprate_knn   | Overall | 17.55  |        2.51        |
|    triage_o2sat_knn     | Overall | 98.38  |        2.43        |
|     triage_sbp_knn      | Overall | 134.83 |       22.22        |
|     triage_dbp_knn      | Overall | 77.45  |       14.78        |
|     triage_pain_knn     | Overall |  3.99  |        3.71        |
| ed_temperature_last_knn | Overall | 36.76  |        0.37        |
|  ed_heartrate_last_knn  | Overall | 78.25  |       14.45        |
|  ed_resprate_last_knn   | Overall | 17.27  |        2.48        |
|    ed_o2sat_last_knn    | Overall | 98.13  |  

In [83]:
# List of categorical variables
categorical_vars = ["gender", "med_event", "microbio_event"]

# Create a table to store the results
table = []

# Calculate counts and percentages for each categorical variable
for variable in categorical_vars:
    # Calculate total counts
    total_counts = df_master[variable].count()

    # Calculate overall counts
    overall_counts = df_master[variable].value_counts()
    overall_percentage = overall_counts / overall_counts.sum() * 100

    # Display the results
    print(f"Counts and Percentages for {variable} Overall:")
    print(f"Total Counts: {total_counts}")

    # Format Overall Counts and Percentages
    print("Overall Counts:")
    print(overall_counts)
    print("Overall Percentages:")
    print(overall_percentage.apply(lambda x: f"{x:.2f}%"))

    print("\n" + "="*40 + "\n")

    # Append results to the table
    table.append([variable, "Overall", f"{overall_counts}", f"{overall_percentage}"])

# Display the table
headers = ["Variable", "Outcome", "Counts", "Percentages"]
print(tabulate(table, headers=headers, tablefmt="pretty"))


Counts and Percentages for gender Overall:
Total Counts: 418100
Overall Counts:
gender
F    227007
M    191093
Name: count, dtype: int64
Overall Percentages:
gender
F    54.29%
M    45.71%
Name: count, dtype: object


Counts and Percentages for med_event Overall:
Total Counts: 418100
Overall Counts:
med_event
1.0    305273
0.0    112827
Name: count, dtype: int64
Overall Percentages:
med_event
1.0    73.01%
0.0    26.99%
Name: count, dtype: object


Counts and Percentages for microbio_event Overall:
Total Counts: 418100
Overall Counts:
microbio_event
0.0    344824
1.0     73276
Name: count, dtype: int64
Overall Percentages:
microbio_event
0.0    82.47%
1.0    17.53%
Name: count, dtype: object


+----------------+---------+---------------------------+-----------------------------+
|    Variable    | Outcome |          Counts           |         Percentages         |
+----------------+---------+---------------------------+-----------------------------+
|     gender     | Overall |        

In [17]:
#Overall cases split

# Count the number of Hospitalization and discharge cases
critical_cases = df_master['outcome_hospitalization'].sum()
non_critical_cases = len(df_master) - critical_cases

# Print the results
print(f"Hospitalization Cases: {critical_cases}")
print(f"Discharge Cases: {non_critical_cases}")

# Count the number of critical and non-critical cases
critical_cases = df_master['outcome_critical'].sum()
non_critical_cases = len(df_master) - critical_cases

# Print the results
print(f"Critical Cases: {critical_cases}")
print(f"Non-Critical Cases: {non_critical_cases}")


# Count the number of 72-hr Reattendance and Non-72-hr Reattendance Cases cases
critical_cases = df_master['outcome_ed_revisit_3d'].sum()
non_critical_cases = len(df_master) - critical_cases

# Print the results
print(f"72-hr Reattendance Cases: {critical_cases}")
print(f"Non-72-hr Reattendance Cases: {non_critical_cases}")

# Filter the df_master dataset to exclude hospitalized cases
df_master_filtered = df_master[df_master['outcome_hospitalization'] == False]

# Count the number of cases with 72-hour reattendance in the filtered dataset
cases_72hr_reattendance = df_master_filtered['outcome_ed_revisit_3d'].sum()

# Count the number of cases without the outcome
cases_without_outcome = len(df_master_filtered) - cases_72hr_reattendance

# Print the results
print(f"Number of cases with 72-hour reattendance (excluding hospitalized cases): {cases_72hr_reattendance}")
print(f"Number of cases without the outcome: {cases_without_outcome}")





Hospitalization Cases: 197824
Discharge Cases: 220276
Critical Cases: 24899
Non-Critical Cases: 393201
72-hr Reattendance Cases: 14972
Non-72-hr Reattendance Cases: 403128
Number of cases with 72-hour reattendance (excluding hospitalized cases): 10548
Number of cases without the outcome: 209728


In [104]:
#overall by test/train



# Create a dictionary to store the statistics
statistics = {}

# Calculate statistics for the Training Data
statistics['Training Data'] = {
    'Hospitalization': f"{df_train['outcome_hospitalization'].sum()} ({df_train['outcome_hospitalization'].mean():.2f}%)",
    'Critical Outcome': f"{df_train['outcome_critical'].sum()} ({df_train['outcome_critical'].mean():.2f}%)",
    'ED Reattendance in 72 hours': f"{df_train['outcome_ed_revisit_3d'].sum()} ({df_train['outcome_ed_revisit_3d'].mean():.2f}%)"
}

# Calculate statistics for the Test Data
statistics['Test Data'] = {
    'Hospitalization': f"{df_test['outcome_hospitalization'].sum()} ({df_test['outcome_hospitalization'].mean():.2f}%)",
    'Critical Outcome': f"{df_test['outcome_critical'].sum()} ({df_test['outcome_critical'].mean():.2f}%)",
    'ED Reattendance in 72 hours': f"{df_test['outcome_ed_revisit_3d'].sum()} ({df_test['outcome_ed_revisit_3d'].mean():.2f}%)"
}

# Calculate statistics for the Total (by outcome)
statistics['Total (by outcome)'] = {
    'Hospitalization': f"{df_master['outcome_hospitalization'].sum()} ({df_master['outcome_hospitalization'].mean():.2f}%)",
    'Critical Outcome': f"{df_master['outcome_critical'].sum()} ({df_master['outcome_critical'].mean():.2f}%)",
    'ED Reattendance in 72 hours': f"{df_master['outcome_ed_revisit_3d'].sum()} ({df_master['outcome_ed_revisit_3d'].mean():.2f}%)"
}

# Create a DataFrame from the statistics dictionary
df_statistics = pd.DataFrame(statistics).transpose()


df_statistics


,Hospitalization,Critical Outcome,ED Reattendance in 72 hours
Training Data,158267 (0.47%),19985 (0.06%),11940 (0.04%)
Test Data,39557 (0.47%),4914 (0.06%),3032 (0.04%)
Total (by outcome),197824 (0.47%),24899 (0.06%),14972 (0.04%)


## Hospitalization outcome

### Continous Variables

In [56]:
continuous_vars = ["age", "triage_temperature_knn", "triage_heartrate_knn", "triage_resprate_knn",
                    "triage_o2sat_knn", "triage_sbp_knn", "triage_dbp_knn", "triage_pain_knn",
                    "ed_temperature_last_knn", "ed_heartrate_last_knn", "ed_resprate_last_knn",
                    "ed_o2sat_last_knn", "ed_sbp_last_knn", "ed_dbp_last_knn", "ed_los_hours",
                       "n_ed_30d", "n_ed_90d", "n_ed_365d", "n_hosp_30d", "n_hosp_90d",
                    "n_hosp_365d", "n_icu_30d", "n_icu_90d", "n_icu_365d", "triage_acuity",]

# Create a table to store the results
table = []

# Calculate mean and standard deviation for each continuous variable
for variable in continuous_vars:
    # Calculate mean and standard deviation for ED Revisit cases
    outcome_hospitalization_mean = df_master[df_master['outcome_hospitalization'] == 1][variable].mean()
    outcome_hospitalization_std = df_master[df_master['outcome_hospitalization'] == 1][variable].std()

    # Calculate mean and standard deviation for No ED Revisit cases
    no_outcome_hospitalization_mean = df_master[df_master['outcome_hospitalization'] == 0][variable].mean()
    no_outcome_hospitalization_std = df_master[df_master['outcome_hospitalization'] == 0][variable].std()

    # Append results to the table
    table.append([variable, "Hospitalization", f"{outcome_hospitalization_mean:.2f}", f"{outcome_hospitalization_std:.2f}"])
    table.append([variable, "No Hospitalization", f"{no_outcome_hospitalization_mean:.2f}", f"{no_outcome_hospitalization_std:.2f}"])

# Display the table
headers = ["Variable", "Outcome", "Mean", "Standard Deviation"]
print(tabulate(table, headers=headers, tablefmt="pretty"))

+-------------------------+--------------------+--------+--------------------+
|        Variable         |      Outcome       |  Mean  | Standard Deviation |
+-------------------------+--------------------+--------+--------------------+
|           age           |  Hospitalization   | 60.07  |       19.49        |
|           age           | No Hospitalization | 46.33  |       19.37        |
| triage_temperature_knn  |  Hospitalization   | 36.75  |        0.59        |
| triage_temperature_knn  | No Hospitalization | 36.68  |        0.49        |
|  triage_heartrate_knn   |  Hospitalization   | 86.27  |       18.64        |
|  triage_heartrate_knn   | No Hospitalization | 83.89  |       16.33        |
|   triage_resprate_knn   |  Hospitalization   | 17.84  |        2.85        |
|   triage_resprate_knn   | No Hospitalization | 17.29  |        2.11        |
|    triage_o2sat_knn     |  Hospitalization   | 97.92  |        2.75        |
|    triage_o2sat_knn     | No Hospitalization | 98.

### Categorical Variables

In [59]:
# List of categorical variables
categorical_vars = ["gender", "med_event", "microbio_event"]

# Calculate counts and percentages for each categorical variable
for variable in categorical_vars:
    # Calculate total counts
    total_counts = df_master[variable].count()

    # Calculate counts for ED Revisit cases
    hospitalization_counts = df_master[df_master['outcome_hospitalization'] == 1][variable].value_counts()
    hospitalization_percentage = hospitalization_counts / hospitalization_counts.sum() * 100


    # Calculate counts for No ED Revisit cases
    no_hospitalization_counts = df_master[df_master['outcome_hospitalization'] == 0][variable].value_counts()
    no_hospitalization_counts_percentage = no_hospitalization_counts / no_hospitalization_counts.sum() * 100

    # Display the results
    print(f"Counts and Percentages for {variable} in Hospitalization Cases:")
    print(f"Total Counts: {total_counts}")
    
    # Format Hospitalization Counts and Percentages
    print("Hospitalization Counts:")
    print(hospitalization_counts)
    print("Hospitalization Percentages:")
    print(hospitalization_percentage.apply(lambda x: f"{x:.2f}%"))
    
    # Format No ED Revisit Counts and Percentages
    print("No Hospitalization Counts:")
    print(no_hospitalization_counts)
    print("No Hospitalization Percentages:")
    print(no_hospitalization_counts_percentage.apply(lambda x: f"{x:.2f}%"))
    
    print("\n" + "="*40 + "\n")

Counts and Percentages for gender in Hospitalization Cases:
Total Counts: 418100
Hospitalization Counts:
gender
F    100252
M     97572
Name: count, dtype: int64
ED Hospitalization Percentages:
gender
F    50.68%
M    49.32%
Name: count, dtype: object
No Hospitalization Counts:
gender
F    126755
M     93521
Name: count, dtype: int64
No Hospitalization Percentages:
gender
F    57.54%
M    42.46%
Name: count, dtype: object


Counts and Percentages for med_event in Hospitalization Cases:
Total Counts: 418100
Hospitalization Counts:
med_event
1.0    160220
0.0     37604
Name: count, dtype: int64
ED Hospitalization Percentages:
med_event
1.0    80.99%
0.0    19.01%
Name: count, dtype: object
No Hospitalization Counts:
med_event
1.0    145053
0.0     75223
Name: count, dtype: int64
No Hospitalization Percentages:
med_event
1.0    65.85%
0.0    34.15%
Name: count, dtype: object


Counts and Percentages for microbio_event in Hospitalization Cases:
Total Counts: 418100
Hospitalization Counts:


## Critical Outcome

### Continous Variables

In [64]:
continuous_vars = ["age", "triage_temperature_knn", "triage_heartrate_knn", "triage_resprate_knn",
                    "triage_o2sat_knn", "triage_sbp_knn", "triage_dbp_knn", "triage_pain_knn",
                    "ed_temperature_last_knn", "ed_heartrate_last_knn", "ed_resprate_last_knn",
                    "ed_o2sat_last_knn", "ed_sbp_last_knn", "ed_dbp_last_knn", "ed_los_hours",
                       "n_ed_30d", "n_ed_90d", "n_ed_365d", "n_hosp_30d", "n_hosp_90d",
                    "n_hosp_365d", "n_icu_30d", "n_icu_90d", "n_icu_365d", "triage_acuity",]

# Create a table to store the results
table = []

# Calculate mean and standard deviation for each continuous variable
for variable in continuous_vars:
    # Calculate mean and standard deviation for ED Revisit cases
    outcome_critical_mean = df_master[df_master['outcome_critical'] == 1][variable].mean()
    outcome_critical_std = df_master[df_master['outcome_critical'] == 1][variable].std()

    # Calculate mean and standard deviation for No ED Revisit cases
    no_outcome_critical_mean = df_master[df_master['outcome_critical'] == 0][variable].mean()
    no_outcome_critical_std = df_master[df_master['outcome_critical'] == 0][variable].std()

    # Append results to the table
    table.append([variable, "Critical", f"{outcome_critical_mean:.2f}", f"{outcome_critical_std:.2f}"])
    table.append([variable, "Not Critical", f"{no_outcome_critical_mean:.2f}", f"{no_outcome_critical_std:.2f}"])

# Display the table
headers = ["Variable", "Outcome", "Mean", "Standard Deviation"]
print(tabulate(table, headers=headers, tablefmt="pretty"))

+-------------------------+--------------+--------+--------------------+
|        Variable         |   Outcome    |  Mean  | Standard Deviation |
+-------------------------+--------------+--------+--------------------+
|           age           |   Critical   | 65.44  |       17.85        |
|           age           | Not Critical | 52.04  |       20.51        |
| triage_temperature_knn  |   Critical   | 36.77  |        0.67        |
| triage_temperature_knn  | Not Critical | 36.71  |        0.53        |
|  triage_heartrate_knn   |   Critical   | 90.55  |       21.28        |
|  triage_heartrate_knn   | Not Critical | 84.67  |       17.17        |
|   triage_resprate_knn   |   Critical   | 18.82  |        4.39        |
|   triage_resprate_knn   | Not Critical | 17.47  |        2.31        |
|    triage_o2sat_knn     |   Critical   | 97.22  |        3.67        |
|    triage_o2sat_knn     | Not Critical | 98.46  |        2.30        |
|     triage_sbp_knn      |   Critical   | 129.04 |

### Categorical Variables

In [75]:
# List of categorical variables
categorical_vars = ["gender", "med_event", "microbio_event"]

# Calculate counts and percentages for each categorical variable
for variable in categorical_vars:
    # Calculate total counts
    total_counts = df_master[variable].count()

    # Calculate counts for ED Revisit cases
    critical_counts = df_master[df_master['outcome_critical'] == 1][variable].value_counts()
    critical_percentage = critical_counts / critical_counts.sum() * 100


    # Calculate counts for No ED Revisit cases
    no_critical_counts = df_master[df_master['outcome_critical'] == 0][variable].value_counts()
    no_critical_counts_percentage = no_critical_counts / no_critical_counts.sum() * 100

    # Display the results
    print(f"Counts and Percentages for {variable} in critical Cases:")
    print(f"Total Counts: {total_counts}")
    
    # Format critical Counts and Percentages
    print("Critical Counts:")
    print(critical_counts)
    print("Critical Percentages:")
    print(critical_percentage.apply(lambda x: f"{x:.2f}%"))
    
    # Format No ED Revisit Counts and Percentages
    print("Not critical Counts:")
    print(no_critical_counts)
    print("Not critical Percentages:")
    print(no_critical_counts_percentage.apply(lambda x: f"{x:.2f}%"))
    
    print("\n" + "="*40 + "\n")

Counts and Percentages for gender in critical Cases:
Total Counts: 418100
Critical Counts:
gender
M    13353
F    11546
Name: count, dtype: int64
Critical Percentages:
gender
M    53.63%
F    46.37%
Name: count, dtype: object
Not critical Counts:
gender
F    215461
M    177740
Name: count, dtype: int64
Not critical Percentages:
gender
F    54.80%
M    45.20%
Name: count, dtype: object


Counts and Percentages for med_event in critical Cases:
Total Counts: 418100
Critical Counts:
med_event
1.0    18085
0.0     6814
Name: count, dtype: int64
Critical Percentages:
med_event
1.0    72.63%
0.0    27.37%
Name: count, dtype: object
Not critical Counts:
med_event
1.0    287188
0.0    106013
Name: count, dtype: int64
Not critical Percentages:
med_event
1.0    73.04%
0.0    26.96%
Name: count, dtype: object


Counts and Percentages for microbio_event in critical Cases:
Total Counts: 418100
Critical Counts:
microbio_event
1.0    19576
0.0     5323
Name: count, dtype: int64
Critical Percentages:
m

## 72-}Hr Reattendance


### Continous Variables

In [47]:
continuous_vars = ["age", "triage_temperature_knn", "triage_heartrate_knn", "triage_resprate_knn",
                    "triage_o2sat_knn", "triage_sbp_knn", "triage_dbp_knn", "triage_pain_knn",
                    "ed_temperature_last_knn", "ed_heartrate_last_knn", "ed_resprate_last_knn",
                    "ed_o2sat_last_knn", "ed_sbp_last_knn", "ed_dbp_last_knn", "ed_los_hours",
                       "n_ed_30d", "n_ed_90d", "n_ed_365d", "n_hosp_30d", "n_hosp_90d",
                    "n_hosp_365d", "n_icu_30d", "n_icu_90d", "n_icu_365d", "triage_acuity",]

# Create a table to store the results
table = []

# Calculate mean and standard deviation for each continuous variable
for variable in continuous_vars:
    # Calculate mean and standard deviation for ED Revisit cases
    ed_revisit_mean = df_master[df_master['outcome_ed_revisit_3d'] == 1][variable].mean()
    ed_revisit_std = df_master[df_master['outcome_ed_revisit_3d'] == 1][variable].std()

    # Calculate mean and standard deviation for No ED Revisit cases
    no_ed_revisit_mean = df_master[df_master['outcome_ed_revisit_3d'] == 0][variable].mean()
    no_ed_revisit_std = df_master[df_master['outcome_ed_revisit_3d'] == 0][variable].std()

    # Append results to the table
    table.append([variable, "ED Revisit", f"{ed_revisit_mean:.2f}", f"{ed_revisit_std:.2f}"])
    table.append([variable, "No ED Revisit", f"{no_ed_revisit_mean:.2f}", f"{no_ed_revisit_std:.2f}"])

# Display the table
headers = ["Variable", "Outcome", "Mean", "Standard Deviation"]
print(tabulate(table, headers=headers, tablefmt="pretty"))

+-------------------------+---------------+--------+--------------------+
|        Variable         |    Outcome    |  Mean  | Standard Deviation |
+-------------------------+---------------+--------+--------------------+
|           age           |  ED Revisit   | 50.63  |       18.69        |
|           age           | No ED Revisit | 52.92  |       20.67        |
| triage_temperature_knn  |  ED Revisit   | 36.69  |        0.51        |
| triage_temperature_knn  | No ED Revisit | 36.71  |        0.54        |
|  triage_heartrate_knn   |  ED Revisit   | 86.88  |       16.94        |
|  triage_heartrate_knn   | No ED Revisit | 84.95  |       17.52        |
|   triage_resprate_knn   |  ED Revisit   | 17.42  |        2.16        |
|   triage_resprate_knn   | No ED Revisit | 17.56  |        2.52        |
|    triage_o2sat_knn     |  ED Revisit   | 98.38  |        2.52        |
|    triage_o2sat_knn     | No ED Revisit | 98.38  |        2.42        |
|     triage_sbp_knn      |  ED Revisi

### Categorical Variables

In [37]:
# List of categorical variables
categorical_vars = ["gender", "med_event", "microbio_event"]

# Calculate counts and percentages for each categorical variable
for variable in categorical_vars:
    # Calculate total counts
    total_counts = df_master[variable].count()

    # Calculate counts for ED Revisit cases
    ed_revisit_counts = df_master[df_master['outcome_ed_revisit_3d'] == 1][variable].value_counts()
    ed_revisit_percentage = ed_revisit_counts / ed_revisit_counts.sum() * 100


    # Calculate counts for No ED Revisit cases
    no_ed_revisit_counts = df_master[df_master['outcome_ed_revisit_3d'] == 0][variable].value_counts()
    no_ed_revisit_percentage = no_ed_revisit_counts / no_ed_revisit_counts.sum() * 100

    # Display the results
    print(f"Counts and Percentages for {variable} in ED Revisit Cases:")
    print(f"Total Counts: {total_counts}")
    
    # Format ED Revisit Counts and Percentages
    print("ED Revisit Counts:")
    print(ed_revisit_counts)
    print("ED Revisit Percentages:")
    print(ed_revisit_percentage.apply(lambda x: f"{x:.2f}%"))
    
    # Format No ED Revisit Counts and Percentages
    print("No ED Revisit Counts:")
    print(no_ed_revisit_counts)
    print("No ED Revisit Percentages:")
    print(no_ed_revisit_percentage.apply(lambda x: f"{x:.2f}%"))
    
    print("\n" + "="*40 + "\n")

Counts and Percentages for gender in ED Revisit Cases:
Total Counts: 418100
ED Revisit Counts:
gender
M    8002
F    6970
Name: count, dtype: int64
ED Revisit Percentages:
gender
M    53.45%
F    46.55%
Name: count, dtype: object
No ED Revisit Counts:
gender
F    220037
M    183091
Name: count, dtype: int64
No ED Revisit Percentages:
gender
F    54.58%
M    45.42%
Name: count, dtype: object


Counts and Percentages for med_event in ED Revisit Cases:
Total Counts: 418100
ED Revisit Counts:
med_event
1.0    9606
0.0    5366
Name: count, dtype: int64
ED Revisit Percentages:
med_event
1.0    64.16%
0.0    35.84%
Name: count, dtype: object
No ED Revisit Counts:
med_event
1.0    295667
0.0    107461
Name: count, dtype: int64
No ED Revisit Percentages:
med_event
1.0    73.34%
0.0    26.66%
Name: count, dtype: object


Counts and Percentages for microbio_event in ED Revisit Cases:
Total Counts: 418100
ED Revisit Counts:
microbio_event
0.0    14515
1.0      457
Name: count, dtype: int64
ED Revi